In [ ]:
import os
import json
import fsspec

import pandas as pd

with fsspec.open(
    "az://carbonplan-scratch/overcredited_arbocs.json",
    account_key=os.environ["BLOB_ACCOUNT_KEY"],
    account_name="carbonplan",
    mode="r",
) as f:
    d = json.load(f)


with fsspec.open(
    "az://carbonplan-retro/remapping/overcredited_arbocs.json",
    account_key=os.environ["BLOB_ACCOUNT_KEY"],
    account_name="carbonplan",
    mode="r",
) as f:
    d = json.load(f)

import seaborn as sns
from carbonplan import styles
import matplotlib.pyplot as plt

styles.mpl.set_theme(style="carbonplan_light", font_scale=1.5)
colors = styles.colors.light


import numpy as np

from carbonplan_retro.data import cat

retro_json = cat.retro_db_light_json.read()

In [ ]:
def get_portfolio_crediting_errors(fn):
    with fsspec.open(
        fn,
        account_key=os.environ["BLOB_ACCOUNT_KEY"],
        account_name="carbonplan",
        mode="r",
    ) as f:

        d = json.load(f)

    # this can go once we've run things through again
    corrected_oc = {}
    for k, v in d.items():
        calc_arbocs = [
            project["arbocs"]["calculated"] for project in retro_json if project["opr_id"] == k
        ][0]
        corrected_oc[k] = [calc_arbocs - alt_est for alt_est in v["alt_arbocs"]]
    df = pd.DataFrame({k: v for k, v in corrected_oc.items() if k != "ACR360"})
    # df = pd.DataFrame({k: v["delta_arbocs"] for k, v in d.items() if k != "ACR360"})
    return df

In [ ]:
res = get_portfolio_crediting_errors(
    "az://carbonplan-retro/results/reclassification-crediting-error.json"
)

In [ ]:
with fsspec.open(
    "az://carbonplan-retro/results/reclassification-crediting-error.json",
    account_key=os.environ["BLOB_ACCOUNT_KEY"],
    account_name="carbonplan",
    mode="r",
) as f:

    d = json.load(f)

del d["CAR1102"]
del d["ACR360"]

In [ ]:
np.sum([np.percentile(d[k]["delta_arbocs"], 50) for k in d.keys()])

In [ ]:
res["CAR1205"]

In [ ]:
res.sum(axis=1).describe(percentiles=[0.05, 0.95]).round()

In [ ]:
from sklearn.metrics import mean_absolute_error

In [ ]:
remapped = get_portfolio_crediting_errors(
    "az://carbonplan-retro/remapping/overcredited_arbocs.json",
)

In [ ]:
classify_compare = pd.concat(
    [remapped.median(axis=0).rename("remapped"), res.median(axis=0).rename("reclass")], axis=1
).dropna()

In [ ]:
(classify_compare["remapped"] - classify_compare["reclass"]).abs().median()

In [ ]:
mean_absolute_error(classify_compare["remapped"], classify_compare["reclass"])

In [ ]:
import seaborn as sns

sns.lmplot("new", "old", data=ij.round(1) / 1_000_000, fit_reg=False)

In [ ]:
reclass["ACR255"].describe().round()

In [ ]:
remapped = get_portfolio_crediting_errors(
    "az://carbonplan-retro/remapping/overcredited_arbocs.json"
)

In [ ]:
comp = pd.concat(
    [reclass.median(axis=0).rename("reclass"), remapped.median(axis=0).rename("remap")], axis=1
)

In [ ]:
comp[(comp["reclass"] > 0) & (comp["remap"] < 0)].round(1)

In [ ]:
project = [project for project in retro_json if project["opr_id"] == "ACR255"][0]

In [ ]:
from collections import Counter, defaultdict

In [ ]:
aa_frac = defaultdict(Counter)
for k in project["assessment_areas"]:
    for species in k["species"]:
        aa_frac[k["code"]][species["code"]] += species["fraction"]

In [ ]:
list(set([species["code"] for k in project["assessment_areas"] for species in k["species"]]))

In [ ]:
project["assessment_areas"]

In [ ]:
aa_frac

In [ ]:
import seaborn as sns

sns.lmplot(
    data=comp,
    x="remap",
    y="reclass",
    fit_reg=False,
    scatter_kws={"color": ".3", "alpha": 0.55, "s": 70},
)

In [ ]:
remapped.median(axis=0)

In [ ]:
df = pd.DataFrame({k: v for k, v in corrected_oc.items() if k != "ACR360"})

In [ ]:
df = pd.DataFrame({k: v["delta_arbocs"] for k, v in d.items() if k != "ACR360"})

In [ ]:
pd.Series(d["ACR189"]["delta_arbocs"]).describe(percentiles=[0.05, 0.95]).round()

In [ ]:
pd.Series(d["ACR427"]["delta_arbocs"]).describe().round(2)

In [ ]:
pd.Series(d["ACR427"]["delta_arbocs"]).describe().round(2)

In [ ]:
df.shape

In [ ]:
df.sum(axis=1).describe(percentiles=[0.05, 0.95]).round(1)

In [ ]:
df.sum(axis=1).describe(percentiles=[0.05, 0.95]).round(1)

In [ ]:
(df.sum(axis=1) / 1_000_000).hist(color=".3", alpha=0.55, bins=25)
plt.grid(False)
plt.xlabel("Overcrediting (Million ARBOCs)")
plt.ylabel("Draw Count")
plt.savefig("/home/jovyan/pub-figs/distribution_overcrediting.pdf", dpi=300, bbox_inches="tight")

In [ ]:
from collections import Counter

In [ ]:
analyzed = df.columns.tolist()

In [ ]:
len([p["opr_id"] for p in retro_json if p["opr_id"] not in analyzed])

In [ ]:
largest_aa = {}
for project in retro_json:
    try:
        acreage_counter = Counter()
        for assessment_area in project["assessment_areas"]:
            if assessment_area["code"] != 999:
                acreage_counter[assessment_area["code"]] += assessment_area["acreage"]
        largest_aa[project["opr_id"]] = max(acreage_counter, key=acreage_counter.get)
    except:
        print(project["opr_id"])

In [ ]:
arb_stats = pd.read_csv("/home/jovyan/lost+found/cleaned_olaf.csv")

In [ ]:
arboc_range = df.quantile(q=[0.25, 0.75]).T

In [ ]:
arboc_range["norm_range"] = (arboc_range[0.75] - arboc_range[0.25]) / (
    arboc_range[0.25] + arboc_range[0.75]
)

In [ ]:
arb_nconds = arb_stats.groupby("aa_code")["cond_prop_group"].sum()

arb_stats["cov"] = arb_stats["slag_co2e_std"] / arb_stats["slag_co2e_acre"]

arboc_range["nconds"] = arboc_range.reset_index().index.map(arb_nconds)

subset = arboc_range[arboc_range["nconds"] < 1500]

# plt.scatter(arboc_range['nconds'], arboc_range['norm_range'], )
plt.scatter(subset["nconds"], subset["norm_range"], c=".3", alpha=0.55, s=125)
plt.xlabel("ARB Reported\nFIA Condition Count")
plt.ylabel("Normalized\nOvercrediting IQR")

In [ ]:
plt.scatter(
    arboc_range["nconds"],
    arboc_range["norm_range"],
)
df.T.loc[nf_opr_ids].T.median(axis=0).round(0)

In [ ]:
nf_opr_ids = [project["opr_id"] for project in nf_proj]

In [ ]:
issued_arbocs = pd.Series(
    {project["opr_id"]: project["arbocs"]["issuance"] for project in retro_json}
).rename("issuance")

In [ ]:
df = pd.concat([df.quantile(0.5, axis=0).rename("overcredit"), issued_arbocs], axis=1).dropna()